In [1]:
import xsimlab as xs


@xs.process
class AdvectionLax1D:
    """Wrap 1-dimensional advection in a single Process."""

    spacing = xs.variable(description="grid spacing", static=True)
    length = xs.variable(description="grid total length", static=True)
    x = xs.variable(dims="x", intent="out")

    v = xs.variable(dims=[(), "x"], description="velocity")

    loc = xs.variable(description="location of initial profile", static=True)
    scale = xs.variable(description="scale of initial profile", static=True)
    u = xs.variable(
        dims="x", intent="out", description="quantity u", attrs={"units": "m"}
    )

    def initialize(self):
        self.x = np.arange(0, self.length, self.spacing)
        self.u = np.exp(-1 / self.scale ** 2 * (self.x - self.loc) ** 2)

    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        factor = (self.v * dt) / (2 * self.spacing)
        u_left = np.roll(self.u, 1)
        u_right = np.roll(self.u, -1)
        self.u1 = 0.5 * (u_right + u_left) - factor * (u_right - u_left)

    def finalize_step(self):
        self.u = self.u1

# below here is the bitch

In [2]:
@xs.process
class UniformGrid1D:
    """Create a 1-dimensional, equally spaced grid."""

    spacing = xs.variable(description="uniform spacing", static=True)
    length = xs.variable(description="total length", static=True)
    
    fuckzou = xs.index(dims='x')
    y = xs.index(dims="fuckzou")

    def initialize(self):
        self.y = np.arange(0, self.length, self.spacing)
        self.fuckzou = np.arange(0, self.length, self.spacing)

In [3]:
@xs.process
class ProfileU:
    """Compute the evolution of the profile of quantity `u`."""

    u_vars = xs.group("u_vars")
    u = xs.variable(
        dims="fuckzou", intent="inout", description="quantity u", attrs={"units": "m"}
    )

    def run_step(self):
        self._delta_u = sum((v for v in self.u_vars))

    def finalize_step(self):
        self.u += self._delta_u

In [4]:
@xs.process
class AdvectionLax:
    """Advection using finite difference (Lax method) on
    a fixed grid with periodic boundary conditions.

    """

    v = xs.variable(dims=[(), "fuckzou"], description="velocity")
    grid_spacing = xs.foreign(UniformGrid1D, "spacing")
    u = xs.foreign(ProfileU, "u")
    u_advected = xs.variable(dims="fuckzou", intent="out", groups="u_vars")

    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        factor = self.v / (2 * self.grid_spacing)

        u_left = np.roll(self.u, 1)
        u_right = np.roll(self.u, -1)
        u_1 = 0.5 * (u_right + u_left) - factor * dt * (u_right - u_left)

        self.u_advected = u_1 - self.u

In [6]:
@xs.process
class InitUGauss:
    """Initialize `u` profile using a Gaussian pulse."""

    loc = xs.variable(description="location of initial pulse", static=True)
    scale = xs.variable(description="scale of initial pulse", static=True)
    #x = xs.foreign(UniformGrid1D, "x")
    fuckzou = xs.foreign(UniformGrid1D, "fuckzou")
    u = xs.foreign(ProfileU, "u", intent="out")

    def initialize(self):
        self.u = np.exp(-1 / self.scale ** 2 * (self.fuckzou - self.loc) ** 2)

In [7]:
advect_model = xs.Model(
    {
        "grid": UniformGrid1D,
        "profile": ProfileU,
        "init": InitUGauss,
        "advect": AdvectionLax,
    }
)

In [8]:
advect_model

<xsimlab.Model (4 processes, 5 inputs)>
grid
    spacing     [in] uniform spacing
    length      [in] total length
init
    loc         [in] location of initial pulse
    scale       [in] scale of initial pulse
advect
    v           [in] () or ('fuckzou',) velocity
profile

In [9]:
import numpy as np
import matplotlib.pyplot as plt
in_ds = xs.create_setup(
     model=advect_model,
     clocks={
        'time': np.linspace(0., 1., 101),
        'otime': [0, 0.5, 1]
    },
    master_clock='time',
    input_vars={
        'grid': {'length': 1.5, 'spacing': 0.01},
        'init': {'loc': 0.3, 'scale': 0.1},
        'advect__v': 1.
     },
      output_vars={
        'profile__u': "otime"}
     )
out_ds = in_ds.xsimlab.run(model=advect_model)

In [10]:
out_ds

<xarray.Dataset>
Dimensions:        (x: 150, otime: 3, fuckzou: 150, time: 101)
Coordinates:
    fuckzou        (x) float64 0.0 0.01 0.02 0.03 0.04 ... 1.46 1.47 1.48 1.49
  * otime          (otime) float64 0.0 0.5 1.0
  * time           (time) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0
Dimensions without coordinates: x
Data variables:
    advect__v      float64 1.0
    grid__length   float64 1.5
    grid__spacing  float64 0.01
    init__loc      float64 0.3
    init__scale    float64 0.1
    profile__u     (otime, fuckzou) float64 0.0001234 0.0002226 ... 0.02705
    y              (fuckzou) float64 0.0 0.01 0.02 0.03 ... 1.46 1.47 1.48 1.49

In [59]:
in_ds

<xarray.Dataset>
Dimensions:        (time: 101, otime: 3)
Coordinates:
  * time           (time) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0
  * otime          (otime) float64 0.0 0.5 1.0
Data variables:
    grid__length   float64 1.5
    grid__spacing  float64 0.01
    init__loc      float64 0.3
    init__scale    float64 0.1
    advect__v      float64 1.0

In [60]:
out_ds.x

<xarray.DataArray 'x' (x: 150)>
array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149])
Dimensions without coordinates: x

In [22]:
xs.variable_info(UniformGrid1D, "x")

No description given

Variable properties:

- type : ``index``
- intent : ``out``
- dimensions : ('x',)


In [12]:
xs.process_info(UniformGrid1D)

<UniformGrid1D  (xsimlab process)>
Variables:
    spacing     [in] uniform spacing
    length      [in] total length
    x          [out] ('x',) 
Simulation stages:
    initialize
